In [1]:
import pandas as pd
import matplotlib.pyplot as plt  
import numpy as np 
import seaborn as sns 

/home/utilisateur/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/utilisateur/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
/tmp/ipykernel_5589/2201431319.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [84]:
data = pd.read_csv('SBAnational.csv')

/tmp/ipykernel_5589/1533426435.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('SBAnational.csv')


##### On crée un nouveau dataframe où les colonnes qui sont inutiles pour les prédictions sont supprimées ainsi que les variables suspectées de faire du dataleaking.

In [85]:
data_lite = data.drop(['BalanceGross', 'NoEmp', 'LoanNr_ChkDgt', 'Name', 'Zip', 'City', 'ChgOffDate', 'DisbursementDate', 'DisbursementGross', 'BankState', 'Bank', 'State', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv', 'CreateJob', 'RetainedJob'], axis=1)
data_lite.head(5)

,NAICS,ApprovalDate,ApprovalFY,Term,NewExist,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status
0,451120,28-Feb-97,1997,84,2.0,1,0,N,Y,P I F
1,722410,28-Feb-97,1997,60,2.0,1,0,N,Y,P I F
2,621210,28-Feb-97,1997,180,1.0,1,0,N,N,P I F
3,0,28-Feb-97,1997,60,1.0,1,0,N,Y,P I F
4,0,28-Feb-97,1997,240,1.0,1,0,N,N,P I F


#####  Vérifier les types des variables, les valeurs manquantes ou suspectes.
On vérifie les valeurs uniques pour chaque variable du dataframe ainsi que les 'missing values'.

In [86]:
print(data_lite.isnull().sum())

data_lite.dtypes


NAICS               0
ApprovalDate        0
ApprovalFY          0
Term                0
NewExist          136
FranchiseCode       0
UrbanRural          0
RevLineCr        4528
LowDoc           2582
MIS_Status       1997
dtype: int64


NAICS              int64
ApprovalDate      object
ApprovalFY        object
Term               int64
NewExist         float64
FranchiseCode      int64
UrbanRural         int64
RevLineCr         object
LowDoc            object
MIS_Status        object
dtype: object

Les variables numériques: NAICS, Term, NewExist, FranchiseCode, UrbanRural

Les variables catégorielles: ApprovalDate, ApprovalFY, RevLineCr, LowDoc, MIS_Status (la target).

In [87]:
#pour la colonne ApprovalDate, on va réduire les modalités en ne considérant que l'année
# d'abord transformer la date en Y-M-D

data_lite['ApprovalDate'] = pd.to_datetime(data_lite['ApprovalDate'], format='%d-%b-%y', errors='coerce')
data_lite['ApprovalDate_Year'] = data_lite['ApprovalDate'].dt.year
data_lite = data_lite.drop('ApprovalDate', axis=1)

#On verifie que les colonnes 'ApprovalDateYear' et 'ApprovalFY' sont identiques
data_lite['ApprovalDate_Year'].equals(data_lite['ApprovalFY'])

False

Vérifier les valeurs uniques des colonnes. Si les valeurs différent des modalités officielles, on les remplace par NaN.

In [88]:
data_lite['MIS_Status'].unique() 

# on supprime les valeurs manquantes de la target
data_lite.dropna(subset=['MIS_Status'], inplace=True)

# encodage de la target
data_lite['MIS_Status'] = data_lite['MIS_Status'].map({'P I F': 0, 'CHGOFF': 1})
data_lite.MIS_Status.dtype



dtype('int64')

In [89]:
data_lite['RevLineCr'].unique() # renvoie:array(['N', '0', 'Y', 'T',......, 'Q']

valeurs_odd = [ 'NaN', 'nan','0', 'T','`', ',', '1', 'C', '3', '2', 'R', '7','A', '5', '.', '4', '-', 'Q']
data_lite['RevLineCr'] = data_lite['RevLineCr'].replace(valeurs_odd, np.nan)
data_lite['RevLineCr'].unique()

array(['N', nan, 'Y'], dtype=object)

In [90]:
data_lite['Term'].unique() #valeurs numériques 

array([ 84,  60, 180, 240, 120,  45, 297, 162,  12, 300,  87, 114, 144,
       126,  83, 102,  80, 137,  42,  96, 167,   7,  36,  37,  26, 264,
        72,  24,   5,  54,  66, 161,  71,   4,  93, 288, 108,  10,  13,
        90,  19,  16,   3,  27, 149,  41, 246,  18,  57, 104,  82, 298,
        14,  61, 127,  58,  44,  32,  85,  48,  31, 112,  38,  73,  47,
        11, 134,  15,  79,  53,  39,   6, 255,  55, 133,  95,  35,  59,
        62,  68, 123,  46,  70, 138,  40,  52,  25,  65,  91,   1,  74,
        49, 103,  77,  86,  63,  56,  22,   0,  97,  23,  17,  69,  21,
        43,  89, 276,  92, 183,   2, 132,  34, 131,   9,  78,  99, 129,
       216,   8,  29, 289,  30, 119, 228, 168, 208,  81, 147, 125,  94,
        51, 211,  64, 111, 266,  75, 306,  28, 232, 117, 118, 309, 303,
        98, 191, 116,  76, 113, 292,  88, 166, 244, 176, 258, 203, 231,
       142,  33, 157, 165,  50, 210, 294, 301, 106,  20, 318, 229, 204,
       269, 241, 178, 115, 174, 192,  67, 100, 141, 282, 122, 15

In [91]:
data_lite['NewExist'].unique()

#relever les lignes avec la valeur 0 de ma colonne
rows_with_zero = data_lite.loc[data_lite['NewExist']==0]
rows_with_zero

#remplacer les 0 par des Nan
data_lite['NewExist'] = data_lite['NewExist'].replace(0, np.nan)
data_lite['NewExist'].unique()

array([ 2.,  1., nan])

In [92]:
data_lite['FranchiseCode'].unique()

array([    1,     0, 15100, ...,  2899, 18701, 15930])

In [93]:
data_lite['UrbanRural'].unique() #remplace 0 par NaN

#remplacer les 0 par des Nan
data_lite['UrbanRural'] = data_lite['UrbanRural'].replace(0, np.nan)
data_lite['UrbanRural'].unique()


array([nan,  1.,  2.])

In [94]:
data_lite['LowDoc'].unique()
valeurs_odd = ['C', '1','S', 'R', 'A', '0']
data_lite['LowDoc'] = data_lite['RevLineCr'].replace(valeurs_odd, np.nan)
data_lite['LowDoc'].unique()

array(['N', nan, 'Y'], dtype=object)

In [95]:
data_lite.dtypes

NAICS                  int64
ApprovalFY            object
Term                   int64
NewExist             float64
FranchiseCode          int64
UrbanRural           float64
RevLineCr             object
LowDoc                object
MIS_Status             int64
ApprovalDate_Year      int32
dtype: object

#### Enregistre le dataframe data_lite pour le réutiliser dans le notebook modelisation


In [96]:
data_lite.to_csv('data.csv', index=False)

